In [1]:
!pip install openkiwi
!pip install torchtext==0.8.0 torch==1.7.0 pytorch-lightning==0.8.5


**** Missing build for a loaded module for Python version "3.6.9". 
Directory does not exist: /share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.6/site-packages

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
  Using cached torchtext-0.3.1-py3-none-any.whl (62 kB)
  Using cached PyYAML-3.13-cp36-cp36m-linux_x86_64.whl
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.8.0
    Uninstalling torchtext-0.8.0:
      Successfully uninstalled torchtext-0.8.0
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the followi

  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
openkiwi 0.1.3 requires pyyaml<4.0,>=3.13, but you have pyyaml 6.0 which is incompatible.
openkiwi 0.1.3 requires torchtext<0.4.0,>=0.3.1, but you have torchtext 0.8.0 which is incompatible.


In [2]:
%%capture
!python -m spacy download en
!python -m spacy download de

In [3]:
import spacy

spacy_en = spacy.load('en_core_web_sm') 
spacy_de = spacy.load('de_core_news_sm') 

In [4]:
def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

In [5]:
from torchtext.data import Field, BucketIterator

SRC = Field(tokenize=tokenize_en, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)
TRG = Field(tokenize=tokenize_de, init_token="<sos>", eos_token="<eos>", lower=True, batch_first=True)

/usr4/dl523/guoyf1/.local/lib/python3.6/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [6]:
from torchtext.datasets import Multi30k

train_dataset, valid_dataset, test_dataset = Multi30k.splits(exts=(".en", ".de"), fields=(SRC, TRG))

/usr4/dl523/guoyf1/.local/lib/python3.6/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [7]:
SRC.build_vocab(train_dataset, min_freq=2)
TRG.build_vocab(train_dataset, min_freq=2)

print(f"len(SRC): {len(SRC.vocab)}")
print(f"len(TRG): {len(TRG.vocab)}")

len(SRC): 5893
len(TRG): 7853


In [8]:
import csv
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import pandas as pd
from torch import nn
from torch import optim

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [19]:
def read_annotated_file(path, index="index"):
    indices = []
    originals = []
    translations = []
    z_means = []
    with open(path, mode="r", encoding="utf-8-sig") as csvfile:
        reader = csv.DictReader(csvfile, delimiter="\t", quoting=csv.QUOTE_NONE)
        for row in reader:
            indices.append(row[index])
            originals.append(row["original"])
            translations.append(row["translation"])
            z_means.append(float(row["z_mean"]))

    return pd.DataFrame(
        {'index': indices,
         'original': originals,
         'translation': translations,
         'z_mean': z_means
         })

def read_test_file(path, index="index"):
    indices = []
    originals = []
    translations = []
    with open(path, mode="r", encoding="utf-8-sig") as csvfile:
        reader = csv.DictReader(csvfile, delimiter="\t", quoting=csv.QUOTE_NONE)
        for row in reader:
            indices.append(row[index])
            originals.append(row["original"])
            translations.append(row["translation"])

    return pd.DataFrame(
        {'index': indices,
         'original': originals,
         'translation': translations,
         })

In [20]:
TRAIN_FILE = "train.ende.df.short.tsv"
DEV_FILE = "dev.ende.df.short.tsv"
TEST_FILE = "test20.ende.df.short.tsv"
train_data = read_annotated_file(TRAIN_FILE)
dev_data = read_annotated_file(DEV_FILE)
test_data = read_annotated_file(TEST_FILE)
train_data = train_data[['original', 'translation', 'z_mean']]
dev_data = dev_data[['original', 'translation', 'z_mean']]
test_data = test_data[['original', 'translation', 'z_mean']]

In [21]:
class QEDataset(Dataset):
    def __init__(self, myDataset):
        self.dataset = myDataset
        self.original = myDataset['original']
        self.translation = myDataset['translation']
        self.z_mean = myDataset['z_mean']

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        return (self.original[index], self.translation[index], self.z_mean[index])


class QEDataLoader(DataLoader):
    def __init__(self, data_dir, batch_size):
        dataset = QEDataset(data_dir)
        super().__init__(dataset, batch_size=batch_size, collate_fn=self.collate_fn)
        return

    def collate_fn(self, batch):
        src_batch, trg_batch, score_batch = [], [], []
        for src_sample, trg_sample, score_sample in batch:
            src_batch.append(src_sample)
            trg_batch.append(trg_sample)
            score_batch.append(score_sample)
        # src_batch = pad_sequence(
        #     src_batch, padding_value=PAD_IDX, batch_first=True)
        # trg_batch = pad_sequence(
        #     trg_batch, padding_value=PAD_IDX, batch_first=True)
        # score_batch = torch.tensor(score_batch).unsqueeze(-1)
        return src_batch, trg_batch, score_batch


In [58]:
import Transformer

class Mlp(nn.Module):
    def __init__(self, in_features, hidden_features=None, out_features=None, act_layer=nn.GELU, drop=0.):
        super().__init__()
        out_features = out_features or in_features
        hidden_features = hidden_features or in_features
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = act_layer()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

class TransformerQE(nn.Module):
    def __init__(self, dim_model, dim_hidden, src_dim_vocab, trg_dim_vocab, mlp_hidden, N=6, h=8, dropout=0.1):
        super().__init__()
        self.encoder = Transformer.Encoder(N, dim_model, dim_hidden, h, dropout)
        self.decoder = Transformer.Decoder(N, dim_model, dim_hidden, h, dropout)
        self.src_embedding = Transformer.TokenEmbedding(src_dim_vocab, dim_model)
        self.trg_embedding = Transformer.TokenEmbedding(trg_dim_vocab, dim_model)
        self.enc_positional_encoding = Transformer.PositionalEncoding(dim_model, dropout=dropout)
        self.dec_positional_encoding = Transformer.PositionalEncoding(dim_model, dropout=dropout)
        self.linear1 = nn.Linear(mlp_hidden, mlp_hidden)
        self.output = Mlp(dim_model, mlp_hidden*2, 1, drop=0.1)
        
    def encode(self, X, padding_mask):
        src_emb = self.enc_positional_encoding(self.src_embedding(X))
        return self.encoder(src_emb, padding_mask)

    def decode(self, X, E, src_padding_mask, trg_padding_mask, peek_mask):
        trg_emb = self.dec_positional_encoding(self.trg_embedding(X))
        return self.decoder(trg_emb, E, src_padding_mask, trg_padding_mask, peek_mask)
        
    def forward(self, src, trg, src_padding_mask, trg_padding_mask, peek_mask):
        enc = self.encode(src, src_padding_mask)
        dec = self.decode(trg, enc, src_padding_mask, trg_padding_mask, peek_mask)
        out = self.linear1(dec).mean(dim=1, keepdim=False)
        return self.output(out) # output are logits

In [67]:
batch_size = 128

src_vocab_dim = len(SRC.vocab)
trg_vocab_dim = len(TRG.vocab)
dim_model = 512
dim_hidden = 2048
num_layers = 6
num_heads = 8

# Create data loaders.
train_dataloader = QEDataLoader(train_data, batch_size=batch_size)
dev_dataloader = QEDataLoader(dev_data, batch_size=batch_size)
test_dataloader = QEDataLoader(test_data, batch_size=batch_size)
loss_fn = nn.MSELoss()
model = TransformerQE(dim_model, dim_hidden, src_vocab_dim, trg_vocab_dim, N=num_layers, h=num_heads, dropout=0.1, 
                 mlp_hidden = dim_model).to(device)
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
optimizer = optim.Adam(model.parameters(), lr=0.0001)


In [68]:
# pretrained_dict = torch.load('transformer_best.ckpt', map_location=torch.device('cpu'))
pretrained_dict = torch.load('transformer_best.ckpt')

model_dict = model.state_dict()
# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
# 2. overwrite entries in the existing state dict
model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [69]:
from torchtext.data.utils import get_tokenizer
from torch.nn.utils.rnn import pad_sequence

epoc = 1

src_tok = get_tokenizer('spacy', language='en_core_web_sm')
trg_tok = get_tokenizer('spacy', language='de_core_news_sm')

def process_data(data, tok, voc):
    emb_tok = tok(data.lower())
    emb = [2] + [voc.vocab.stoi[t.lower()] for t in emb_tok] + [3]

    return emb

def train():
    for i in range(epoc):
        epoch_loss = 0.0
        count = 0
        for batch, (src_batch, trg_batch, score_batch) in enumerate(train_dataloader):
            src, trg = [], []
            for sample in src_batch:
                emb = process_data(sample, src_tok, SRC)
                src.append(torch.tensor(emb))
            src = pad_sequence(src, 
                    padding_value=1, batch_first=True).to(device)
            for sample in trg_batch:
                emb = process_data(sample, trg_tok, TRG)
                trg.append(torch.tensor(emb))
            trg = pad_sequence(trg, 
                    padding_value=1, batch_first=True).to(device)
            src_padding_mask = Transformer.get_padding_mask(
                src , 1).to(device)
            trg_padding_mask = Transformer.get_padding_mask(
                trg, 1).to(device)
            peek_mask = Transformer.get_peek_mask(trg).to(device)
            output = model(src,trg,src_padding_mask,trg_padding_mask,peek_mask).view(-1)
            score_batch = torch.Tensor(score_batch)
            score_batch = score_batch.to(device)
            loss = loss_fn(output, score_batch)
            epoch_loss = epoch_loss + loss.item()
            count = count + 1
            if count % 5 == 0:
                print("Loss in the " + str(count) + " batch is " + str(loss.item()))
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print("Loss in the " + str(count) + " batch is " + str(epoch_loss / count))            

if __name__ == '__main__':
    train()


Loss in the 5 batch is 0.6612106561660767
Loss in the 10 batch is 0.7799232006072998
Loss in the 15 batch is 1.001204490661621
Loss in the 20 batch is 0.962153434753418
Loss in the 25 batch is 0.8681401610374451
Loss in the 30 batch is 0.48221537470817566
Loss in the 35 batch is 0.7186139822006226
Loss in the 40 batch is 0.5772583484649658
Loss in the 45 batch is 0.42456138134002686
Loss in the 50 batch is 0.4442644715309143
Loss in the 55 batch is 1.9784554243087769
Loss in the 55 batch is 0.7424852029843764


In [75]:
def test(dataloader, model, loss_fn):
    model.eval()
    epoch_loss = 0
    count = 0
    res = torch.ones(1).to(device)
    
    with torch.no_grad():
        for batch, (src_batch, trg_batch, score_batch) in enumerate(dataloader):
            src, trg = [], []
            for sample in src_batch:
                emb = process_data(sample, src_tok, SRC)
                src.append(torch.tensor(emb))
            src = pad_sequence(src, 
                    padding_value=1, batch_first=True).to(device)
            for sample in trg_batch:
                emb = process_data(sample, trg_tok, TRG)
                trg.append(torch.tensor(emb))
            trg = pad_sequence(trg, 
                    padding_value=1, batch_first=True).to(device)
            src_padding_mask = Transformer.get_padding_mask(
                src , 1).to(device)
            trg_padding_mask = Transformer.get_padding_mask(
                trg, 1).to(device)
            peek_mask = Transformer.get_peek_mask(trg).to(device)
            output = model(src,trg,src_padding_mask,trg_padding_mask,peek_mask).view(-1)
            res = torch.cat((res,output)) 
            # print(output)
            # print(type(output))
            # print(type(score_batch))
            score_batch = torch.Tensor(score_batch).to(device)
            loss = loss_fn(output, score_batch)
            epoch_loss = epoch_loss + loss.item()
            count = count + 1
    print("Loss in the " + str(count) + " batch is " + str(epoch_loss / count))
    return res

res = test(test_dataloader, model,loss_fn)

Loss in the 8 batch is 0.4905144013464451


In [76]:
from scipy import stats

res = res[1:]
res = res.cpu()
res = res.detach().numpy()

pr = stats.pearsonr(res,test_data['z_mean'])
sp = stats.spearmanr(res,test_data['z_mean'])
pr = pr[0]
sp = sp[0]

print(pr)
print(sp)

-0.04633618474428363
-0.04820083773439874
